In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch
import cv2
from time import time, sleep
from sklearn.decomposition import IncrementalPCA

from utils import *

In [2]:
def read_video(filename, nframes=np.inf):
    """
        Read the given number of frames of a video using 
        the opencv library
    """
    frames = []
    cap = cv2.VideoCapture(filename)
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    i = 0
    while cap.isOpened() and i < nframes:
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
            i += 1
        else:
            break
    
    cap.release()
    return np.array(frames), fps, width, height

In [3]:
print('Video loading...')
# Read the video
start = time()
sample_name = 'sample20s_540'
frames, fps, width, height = read_video('data/'+sample_name+'.mp4')
nframes = frames.shape[0]
print(f'Loaded video of {round(nframes/fps, 2)} seconds with quality {width}x{height} in {round(time()-start,2)} seconds')

# Convert to gray scale
print('Transforming the video...')
gray_frames = np.zeros((nframes, height, width), dtype=np.uint8)
for i, frame in enumerate(frames):
    gray_frames[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Create, fit and apply the pca model
pca_model = IncrementalPCA(n_components=50)
start = time()
pca_model.fit(gray_frames.reshape(gray_frames.shape[0], -1))
after_fit = time()
reduced_gray_frames = pca_model.transform(gray_frames.reshape(gray_frames.shape[0], -1))
gray_frames_2 = pca_model.inverse_transform(reduced_gray_frames).reshape((nframes, height, width))
after_transformations = time()
print(f'Fitting of the pca model took {round(after_fit-start,2)} seconds')
print(f'Transformations of the videos took {round(after_transformations-after_fit,2)} seconds')

# Write the videos to disk
writer_gray = cv2.VideoWriter('data/'+sample_name+'_gray.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         fps, (width, height), 0)
writer_gray_after_pca = cv2.VideoWriter('data/'+sample_name+'_gray_after_pca.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         fps, (width, height), 0)
print('Displaying and writting the video...')
for i in range(nframes):
    frame_pca = np.around(gray_frames_2[i]).astype(np.uint8)
    writer_gray.write(gray_frames[i])
    writer_gray_after_pca.write(frame_pca)
    
    cv2.imshow('Video Capture', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
# When everything done, release the video capture and video write objects
writer_after_pca.release()
writer_gray.release()
writer_gray_after_pca.release()

# Closes all the frames
cv2.destroyAllWindows()
print('Done')

Video loading...
Loaded video of 20.0 seconds with quality 960x540 in 1.12 seconds
Transforming the video...
Fitting of the pca model took 126.14 seconds
Transformations of the videos took 2.79 seconds
Displaying and writting the video...
Done


------------------
(20 or 50 components, gray)

Loaded video of 20.0 seconds with quality 960x540 in 1.13 seconds

Fitting of the pca model took 124.42 seconds

Transformations of the videos took 2.3 seconds

------------------

In [4]:
print('Video loading...')
# Read the video
start = time()
sample_name = 'sample5s'
frames, fps, width, height = read_video('data/'+sample_name+'.mp4')
nframes = frames.shape[0]
print(f'Loaded video of {round(nframes/fps, 2)} seconds with quality {width}x{height} in {round(time()-start,2)} seconds')

print('Transforming the video...')
# Create, fit and apply the pca model
pca_model = IncrementalPCA(n_components=50)
start = time()
pca_model.fit(frames.reshape(frames.shape[0], -1))
after_fit = time()
reduced_frames = pca_model.transform(frames.reshape(frames.shape[0], -1))
frames_2 = pca_model.inverse_transform(reduced_frames).reshape((nframes, height, width, 3))
after_transformations = time()
print(f'Fitting of the pca model took {round(after_fit-start,2)} seconds')
print(f'Transformations of the videos took {round(after_transformations-after_fit,2)} seconds')

# Write the videos to disk
writer_after_pca = cv2.VideoWriter('data/'+sample_name+'_after_pca.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         fps, (width, height))

print('Displaying and writting the video...')
frames_2 = np.around(frames_2).astype(np.uint8)
for frame in frames_2:
    writer_after_pca.write(frame)
    
    cv2.imshow('Video Capture', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
# When everything done, release the video capture and video write objects
writer_after_pca.release()

# Closes all the frames
cv2.destroyAllWindows()
print('Done')

Video loading...
Loaded video of 5.0 seconds with quality 960x540 in 0.3 seconds
Transforming the video...
Fitting of the pca model took 94.62 seconds
Transformations of the videos took 2.97 seconds
Displaying and writting the video...
Done


In [4]:
video = VideoIterator('data/sample5s.mp4')
start = time()
pca_model = IncrementalPCA(n_components=50)
for frames in video:
    print(frames.shape)
    pca_model.partial_fit(frames.reshape(frames.shape[0], -1))
frames,_,_,_ = read_video('data/sample5s.mp4')
nframes = frames.shape[0]

reduced_frames = pca_model.transform(frames.reshape(frames.shape[0], -1))
frames_2 = pca_model.inverse_transform(reduced_frames).reshape((nframes, video.height, video.width, 3))
writer_after_pca = cv2.VideoWriter('data/sample5s_after_pca_2.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         video.fps, (video.width, video.height))

frames_2 = np.around(frames_2).astype(np.uint8)
for frame in frames_2:
    writer_after_pca.write(frame)
print(time()-start,'sec')
writer_after_pca.release()

(64, 540, 960, 3)
(56, 540, 960, 3)
127.25194954872131 sec


In [6]:
video = VideoIterator('data/sample20s_540.mp4')
start = time()
pca_model = IncrementalPCA(n_components=32)
for frames in video:
    pca_model.partial_fit(frames.reshape(frames.shape[0], -1))
print(f'Loading via iterator and fitting took {time()-start} sec')
    
frames,_,_,_ = read_video('data/sample5s.mp4')
nframes = frames.shape[0]

reduced_frames = pca_model.transform(frames.reshape(frames.shape[0], -1))
frames_2 = pca_model.inverse_transform(reduced_frames).reshape((nframes, video.height, video.width, 3))
writer_after_pca = cv2.VideoWriter('data/sample20s_after_pca_2.mp4', cv2.VideoWriter_fourcc(*"MJPG"),
                         video.fps, (video.width, video.height))

frames_2 = np.around(frames_2).astype(np.uint8)
for frame in frames_2:
    writer_after_pca.write(frame)
print(time()-start,'sec')
writer_after_pca.release()

Loading via iterator and fitting took 504.9417932033539 sec
508.76675939559937 sec


5 seconds 960x540 in color:
- First technique: 0.3s (load), 95s (fit), 3s (transformations)
- Second technique: total of 127s (loading twice the video)
20 seconds 960x540 in color:
- First technique: kernel dies (RAM is full)
- Second technique: 509s in total, loading via iterator and fitting took 505s (4 times more than the gray version)

In [13]:
# SVD comparison
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

print('Video loading...')
# Read the video
start = time()
sample_name = 'sample20s_540'
frames, fps, width, height = read_video('data_generated/'+sample_name+'.mp4')
nframes = frames.shape[0]
print(f'Loaded video of {round(nframes/fps, 2)} seconds with quality {width}x{height} in {round(time()-start,2)} seconds')

# Convert to gray scale
print('Transforming the video...')
gray_frames = np.zeros((nframes, height, width), dtype=np.uint8)
for i, frame in enumerate(frames):
    gray_frames[i] = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Fit a SVD model
start = time()
u, s, vh = svds(gray_frames.reshape(gray_frames.shape[0], -1).astype(float), k=50)
after_fit = time()
after_transformations = time()
print(f'Fitting of the scipy SVD model took {round(after_fit-start,2)} seconds')
svd_model = TruncatedSVD(n_components=50)
start = time()
u, s, vh = svd_model.fit(gray_frames.reshape(gray_frames.shape[0], -1))
after_fit = time()
after_transformations = time()
print(f'Fitting of the truncated SVD model took {round(after_fit-start,2)} seconds')
start = time()
u, s, vh = randomized_svd(gray_frames.reshape(gray_frames.shape[0], -1), 50)
after_fit = time()
after_transformations = time()
print(f'Fitting of the randomized SVD model took {round(after_fit-start,2)} seconds')

Video loading...
Loaded video of 20.0 seconds with quality 960x540 in 0.82 seconds
Transforming the video...
Fitting of the scipy SVD model took 20.05 seconds
Fitting of the truncated SVD model took 22.52 seconds
Fitting of the randomized SVD model took 18.98 seconds


Numpy's SVD -> MemoryError: Unable to allocate 1.96 TiB for an array with shape (518400, 518400) and data type float64

In [16]:
# Test scalability of iterator
video = VideoIterator('/home/jules/videos/series/rick-and-morty/Rick and Morty s03e01.mkv', duration=np.inf, batch_size=240)
print(f'Length: {video.duration}s\nFrame per second: {video.fps}\nQuality: {video.width}x{video.height}')

start = time()
for frames in video:
    #u, s, vh = randomized_svd(frames.reshape(frames.shape[0], -1), 50)
    #l, axes = np.linalg.eig(frames.reshape(frames.shape[0], -1))
    frames = np.abs(frames ** 2) + 14*frames
print(video.current_frame/video.fps)
time() - start

Length: 1365.0s
Frame per second: 24
Quality: 1280x720
1364.9583333333333


212.51076078414917